# Воронка BIGQUERY

Выгрузка данных по всем переходам

In [ ]:
from google.cloud import bigquery
import pandas as pd
import statistics as st
import re
import warnings
import numpy as np
warnings.filterwarnings("ignore")
client = bigquery.Client.from_service_account_json(r'xxxx.json')

QUERY_GA_PAGES = ("WITH DF_log AS (SELECT * FROM (SELECT date, clientID, pagePath, IFNULL(prev_pagepath, 'Прямой переход') AS prev_pagepath, deviceCategory, hitTimeMoscow FROM (SELECT date, clientID, hits.page.pagePath AS pagePath, LAG(hits.page.pagePath) OVER (PARTITION BY fullVisitorId ORDER BY visitStartTime) AS prev_pagepath, device.deviceCategory, FORMAT_TIMESTAMP('%F %X', TIMESTAMP_MILLIS(visitstarttime*1000 + time), 'Europe/Moscow') as hitTimeMoscow FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '20200113' AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)) AND hits.type = 'PAGE' )) WHERE pagePath LIKE '%roa%' AND prev_pagepath NOT LIKE '%roa%' ORDER BY clientID, hitTimeMoscow), DF_new AS (SELECT clientID, hit_NewUser FROM (SELECT clientID, hit_NewUser, row_number() OVER (PARTITION BY clientID ORDER BY hit_NewUser)  AS NewUser_Range FROM (SELECT clientID, FORMAT_TIMESTAMP('%F %X', TIMESTAMP_MILLIS(visitstarttime*1000 + time), 'Europe/Moscow') as hit_NewUser, FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '20200113' AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)) AND hits.eventInfo.eventCategory = 'DOS2_smsdos2new')) WHERE NewUser_Range = 1), DF_return AS (SELECT clientID, hit_OldUser FROM (SELECT clientID, hit_OldUser, eventCategory, row_number() OVER (PARTITION BY clientID ORDER BY hit_OldUser)  AS OldUser_Range FROM (SELECT clientID, hits.eventInfo.eventCategory, FORMAT_TIMESTAMP('%F %X', TIMESTAMP_MILLIS(visitstarttime*1000 + time), 'Europe/Moscow') as hit_OldUser, FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '20200113' AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)) AND hits.eventInfo.eventCategory LIKE 'DOS2_smsdos2%' AND hits.eventInfo.eventCategory NOT LIKE '%new%')) WHERE OldUser_Range = 1), DF_step1 AS (SELECT clientID, hit_Step1 FROM (SELECT clientID, hit_Step1, eventCategory, row_number() OVER (PARTITION BY clientID ORDER BY hit_Step1)  AS OldUser_Range FROM (SELECT clientID, hits.eventInfo.eventCategory, FORMAT_TIMESTAMP('%F %X', TIMESTAMP_MILLIS(visitstarttime*1000 + time), 'Europe/Moscow') as hit_Step1, FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '20200113' AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)) AND hits.eventInfo.eventCategory LIKE '%step-1%' )) WHERE OldUser_Range = 1) SELECT * FROM DF_log LEFT JOIN DF_new USING(clientID) LEFT JOIN DF_return USING(clientID) LEFT JOIN DF_step1 USING(clientID) ORDER BY clientID, hitTimeMoscow")
QUERY_GA_PAGES_job = client.query(QUERY_GA_PAGES)
df_pages = QUERY_GA_PAGES_job.to_dataframe()

Процессинг данных для того, чтобы разделить в дальнейшем пользователей на сегменты

In [ ]:
df_pages['date'] = pd.to_datetime(df_pages['date'])
df_pages['st'] = df_pages['date'].dt.to_period('W').apply(lambda r: r.start_time).dt.date#.apply(lambda x: '-'.join(str(x)[5:].split('-')[::-1]))
df_pages['en'] = df_pages['date'].dt.to_period('W').apply(lambda r: r.end_time).dt.date#.apply(lambda x: '-'.join(str(x)[5:].split('-')[::-1]))
df_pages['week'] = df_pages['st'].astype(str) + ' / ' + df_pages['en'].astype(str)
df_pages.drop(['st','en'], axis=1, inplace=True)
df_pages['year'] = pd.DatetimeIndex(df_pages['date']).year
df_pages['month'] = pd.DatetimeIndex(df_pages['date']).month

df_pages_no_events = df_pages[(df_pages['hit_NewUser'].isnull())&(df_pages['hit_OldUser'].isnull())&(df_pages['hit_Step1'].isnull())][['clientID', 'date', 'pagePath', 'prev_pagepath', 'deviceCategory', 'week', 'year', 'month']]
df_pages_no_events['type'] = 'no_interaction'

df_pages = df_pages[(~df_pages['hit_NewUser'].isnull())|(~df_pages['hit_OldUser'].isnull())|(~df_pages['hit_Step1'].isnull())]
df_pages['hitTimeMoscow'] = pd.to_datetime(df_pages['hitTimeMoscow'])
df_pages['hit_NewUser'] = pd.to_datetime(df_pages['hit_NewUser']).fillna(pd.to_datetime('1900-01-01 00:00:00'))
df_pages['hit_OldUser'] = pd.to_datetime(df_pages['hit_OldUser']).fillna(pd.to_datetime('1900-01-01 00:00:00'))
df_pages['hit_Step1'] = pd.to_datetime(df_pages['hit_Step1']).fillna(pd.to_datetime('1900-01-01 00:00:00'))


Сегментация

In [ ]:
df_pages['diff_hits-newUser'] = np.where((df_pages['hitTimeMoscow'] >= df_pages['hit_NewUser']), 'После new user', 'До new user')
df_pages['diff_hits-retUser'] = np.where((df_pages['hitTimeMoscow'] >= df_pages['hit_OldUser']), 'После ret user', 'До ret user')
df_pages['diff_hits-step1'] = np.where((df_pages['hitTimeMoscow'] >= df_pages['hit_Step1']), 'После step1', 'До step1')


df_pages_new = df_pages[df_pages['diff_hits-newUser'] == 'До new user'][['clientID', 'date', 'pagePath', 'prev_pagepath', 'deviceCategory', 'week', 'year', 'month']]
df_pages_new['type'] = 'new'

df_pages_returned = df_pages[(df_pages['diff_hits-newUser'] == 'После new user')&(df_pages['diff_hits-retUser'] == 'До ret user')][['clientID', 'date', 'pagePath', 'prev_pagepath', 'deviceCategory', 'week', 'year', 'month']]
df_pages_returned['type'] = 'returned'

df_pages_step1 = df_pages[(df_pages['diff_hits-step1'] == 'До step1')&(~df_pages['clientID'].isin(set(df_pages_new['clientID'])))&(~df_pages['clientID'].isin(set(df_pages_returned['clientID'])))][['clientID', 'date', 'pagePath', 'prev_pagepath', 'deviceCategory', 'week', 'year', 'month']]
df_pages_step1['type'] = 'only step1'



Унификация урлов и нейминг сегментов

In [ ]:
pages_all = pd.concat([df_pages_new,df_pages_returned,df_pages_step1,df_pages_no_events],axis=0)

list_bq=list(pages_all['prev_pagepath'])
unique_match_bq=[]
for tempos_re in list_bq:
    tempos_re1=re.sub(r'((\?|\#|welcome&|utm|csi&|vas&pact|~).+|\#|\?|lp\/3\.0\/|lp\/|undefined)','', str(tempos_re))
    if tempos_re1 != 'Прямой переход':
        if tempos_re1[len(tempos_re1)-1]=='/':
            tempos_re_fin=str(tempos_re1)
        elif tempos_re1[len(tempos_re1)-1]=='/' and tempos_re1[len(tempos_re1)-2]=='/':
            tempos_re_fin=str(tempos_re1)[0:len(str(tempos_re1))-1]
        else:
            tempos_re_fin=str(tempos_re1)+'/'
        tempos_re_fin_1=re.sub(r'\/\/|\/\s\/','/', str(tempos_re_fin))
        tempos_re_fin_2_1=re.sub(r'.ru/ru/','.ru/', str(tempos_re_fin_1))
        tempos_re_fin_2_2=re.sub(r'(\/faq).+|(\/faq)','/faq/', str(tempos_re_fin_2_1))
        tempos_re_fin_2=re.sub(r'(\/event\-).+','/event/',str(tempos_re_fin_2_2))
        tempos_re_fin_2_3=re.sub(r'(\/issuers).+|(\/issuers)','/issuers/', str(tempos_re_fin_2))
        unique_match_bq.append(tempos_re_fin_2_3)
    else:
        unique_match_bq.append('Прямой переход')
pages_all['prev_pagepath'] = unique_match_bq

dict_stages_for_pages = {'only step1': 'Дошел только до заполнения общей информации',
               'new':'Отправил СМС - Новый',
               'returned':'Отправил СМС - Вернувшийся',
               'no_interaction':'Пришел и не взаимодействовал'}

pages_all_gr = pages_all.groupby(['year','week','type','prev_pagepath','deviceCategory'])['clientID'].nunique().to_frame().reset_index()
pages_all_gr.replace({"type": dict_stages_for_pages},inplace=True)


Создание структуры под Metabase

In [ ]:
pages_all_gr1 = pages_all_gr.copy()
pages_all_gr1['type_page'] = pages_all_gr1['type']
pages_all_gr1['type'] = '00. Пришло на ДОС'
pages_all_gr1['agg_type'] = 'Общий (по умолчанию)'

pages_all_gr2 = pages_all_gr[pages_all_gr['type'] == 'Пришел и не взаимодействовал']
pages_all_gr2['type_page'] = 'Ячейка не используется'
pages_all_gr2['type'] = '00.1 Пришло на ДОС и не взаимодействовало ни с одним этапом'
pages_all_gr2['agg_type'] = 'Добавить детальный срез'

pages_all_gr3 = pages_all_gr[pages_all_gr['type'].str.contains('Отправил|Дошел')]
pages_all_gr3['type_page'] = 'Ячейка не используется'
pages_all_gr3['type'] = '01. Заполнило форму общей информации'
pages_all_gr3['agg_type'] = 'Общий (по умолчанию)'

pages_all_gr4 = pages_all_gr[pages_all_gr['type'] == 'Дошел только до заполнения общей информации']
pages_all_gr4['type_page'] = 'Ячейка не используется'
pages_all_gr4['type'] = '01.1 Заполнило форму общей информации, но не отправило СМС'
pages_all_gr4['agg_type'] = 'Добавить детальный срез'

pages_all_gr5 = pages_all_gr[pages_all_gr['type'].str.contains('Отправил')]
pages_all_gr5['type_page'] = 'Ячейка не используется'
pages_all_gr5['type'] = '02. Отправило СМС всего'
pages_all_gr5['agg_type'] = 'Общий (по умолчанию)'

pages_all_gr6 = pages_all_gr[pages_all_gr['type'] == 'Отправил СМС - Новый']
pages_all_gr6['type_page'] = 'Ячейка не используется'
pages_all_gr6['type'] = '02.1 Отправило СМС - новых'
pages_all_gr6['agg_type'] = 'Добавить детальный срез'

pages_all_gr7 = pages_all_gr[pages_all_gr['type'] == 'Отправил СМС - Вернувшийся']
pages_all_gr7['type_page'] = 'Ячейка не используется'
pages_all_gr7['type'] = '02.2 Отправило СМС - вернувшихся'
pages_all_gr7['agg_type'] = 'Добавить детальный срез'


pages_all_gr = pd.concat([pages_all_gr1, pages_all_gr2, pages_all_gr3,pages_all_gr4, pages_all_gr5, pages_all_gr6,pages_all_gr7], axis=0)
pages_all_gr = pages_all_gr[['year','week','type','type_page','prev_pagepath','deviceCategory','clientID','agg_type']]

# Внутренняя воронка и кубовая воронка

Выгрузка данных по всем событиям во внутренней воронке

In [ ]:
import pyodbc
from datetime import datetime
import datetime
import numpy as np

connection_DB_CRM = pyodbc.connect(Trusted_Connection='yes', driver = '{SQL Server}', server = 'xxx' , database = 'xxx')
sql_crm="SELECT * FROM [datamartMarketing].[vwCrmProspect] WHERE date_creation >= '2019.09.01'"
SQL_CRM_all = pd.read_sql(sql_crm,connection_DB_CRM)

QUERY_STAGES = ("SELECT date, clientID, fullVisitorId, visitId, visitStartTime, TIMESTAMP_SECONDS(visitStartTime) as visit_starttime_UTC, hits.time, CONCAT(cast(trafficSource.source as string),cast(trafficSource.medium as string)) as sourcemedium, trafficSource.campaign as campaign, device.deviceCategory as device, hits.eventInfo.eventCategory AS stage FROM `api-open-broker.108613784.ga_sessions_*`, UNNEST(hits) AS hits WHERE _TABLE_SUFFIX BETWEEN '201900901' AND FORMAT_DATE('%Y%m%d',DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)) AND (hits.eventInfo.eventCategory LIKE '%Stage_DOS2%' OR (hits.eventInfo.eventCategory LIKE 'DOS2_%' AND hits.eventInfo.eventCategory LIKE '%eg') OR hits.eventInfo.eventCategory LIKE '%StageDOS_2_docs%') ORDER BY clientID")
QUERY_GA_STAGES = client.query(QUERY_STAGES)
df_stages = QUERY_GA_STAGES.to_dataframe()
df_stages['stage'] = df_stages['stage'].apply(lambda x: x.replace('Stage_DOS2 |',''))

Процессинг данных

In [ ]:
df_stages['date'] = pd.to_datetime(df_stages['date'])
df_stages['st'] = df_stages['date'].dt.to_period('W').apply(lambda r: r.start_time).dt.date#.apply(lambda x: '-'.join(str(x)[5:].split('-')[::-1]))
df_stages['en'] = df_stages['date'].dt.to_period('W').apply(lambda r: r.end_time).dt.date#.apply(lambda x: '-'.join(str(x)[5:].split('-')[::-1]))
df_stages['week'] = df_stages['st'].astype(str) + ' / ' + df_stages['en'].astype(str)
df_stages.drop(['st','en'], axis=1,inplace=True)
df_stages['year'] = pd.DatetimeIndex(df_stages['date']).year
df_stages['month'] = pd.DatetimeIndex(df_stages['date']).month

df_stages.loc[df_stages.stage.str.contains('DOS2_PartReg'), 'stage'] = ' step-2'
df_stages.loc[df_stages.stage.str.contains('StageDOS_2_docs'), 'stage'] = ' step-8'
df_stages.loc[df_stages.stage.str.contains('DOS2_Fullreg'), 'stage'] = ' step-9'
df_stages_time = df_stages[['year','week','fullVisitorId','visitId','time','stage']]


dict_stages = {' step-1':'01. Оставило заявку',
               ' step-2':'02. Подтвердило СМС (новых или вернувшихся)',
               ' step-3':'03. Заполнило анкету (вручную или через Госуслуги)',
               ' step-4':'04. Зарегистрировало ЛК',
               ' step-5':'05. Заполнило риск профиль',
               ' step-6':'06. Загрузило скан паспорта',
               ' step-7':'07. Выбрало тариф/продукт',
               ' step-8':'08. Сформировало документы',
               ' step-9':'09. Подписало документы'}
df_stages.replace({"stage": dict_stages},inplace=True)


df_stages_gr = df_stages.groupby(['year','week','stage','device'])['clientID'].nunique().to_frame().reset_index()
df_stages_gr['type_page'] = 'Ячейка не используется'
df_stages_gr['prev_pagepath'] = 'Ячейка не используется'
df_stages_gr = df_stages_gr[~df_stages_gr['stage'].str.contains('01.|02.')]
df_stages_gr = df_stages_gr[['year','week','stage','type_page','prev_pagepath','device','clientID']]

Получение и процессинг данных из куба по воронкам, построенным по принципам OLDBI и NEWBI

In [ ]:
import win32com.client

office = win32com.client.Dispatch("Excel.Application")
wb = office.Workbooks.Open(r'C:\Users\korpachev\BI_Reports\Coub1_2020_DOS2_backs.xlsx')
wb.RefreshAll()
wb.Save()
wb.Close()
coub = pd.read_excel(r'C:\Users\korpachev\BI_Reports\Coub1_2020_DOS2_backs.xlsx').fillna(0)

clids_all = df_stages[df_stages['stage'] == '09. Подписало документы'][['clientID','device']].rename(columns={'clientID': 'cookie_id'})
guids_all = SQL_CRM_all.merge(clids_all,on='cookie_id',how='left')
guids_all = guids_all[~guids_all['device'].isnull()][['GUID','cookie_id','device']].drop_duplicates()



coub_nd = coub[coub['Дата активации'].str.contains('2019-09|2019-10|2019-11|2019-12|2020-|2021-|2022-',na=False)]
coub_nd['Дата активации'] = pd.to_datetime(coub_nd['Дата активации'])
coub_nd['st'] = coub_nd['Дата активации'].dt.to_period('W').apply(lambda r: r.start_time).dt.date#.apply(lambda x: '-'.join(str(x)[5:].split('-')[::-1]))
coub_nd['en'] = coub_nd['Дата активации'].dt.to_period('W').apply(lambda r: r.end_time).dt.date#.apply(lambda x: '-'.join(str(x)[5:].split('-')[::-1]))
coub_nd['week'] = coub_nd['st'].astype(str) + ' / ' + coub_nd['en'].astype(str)
coub_nd.drop(['st','en'], axis=1,inplace=True)
coub_nd['year'] = pd.DatetimeIndex(coub_nd['Дата активации']).year

coub_nd_gr = coub_nd.groupby(['year','week','GUID']).count().reset_index().merge(guids_all,on='GUID',how='left')
coub_nd_gr = coub_nd_gr[~coub_nd_gr['device'].isnull()].groupby(['year','week','device'])['GUID'].count().reset_index()
coub_nd_gr['stage'] = '10.1 Активировано договоров за этот период всего'
coub_nd_gr = coub_nd_gr[['year','week','stage','device','GUID']]


coub_nk = coub[(coub['Дата'].str.contains('2019-09|2019-10|2019-11|2019-12|2020-|2021-|2022-',na=False))&(coub['К-во персон 5 - 50 т.р.'] >= 1)]
coub_nk['Дата'] = pd.to_datetime(coub_nk['Дата'])
coub_nk['st'] = coub_nk['Дата'].dt.to_period('W').apply(lambda r: r.start_time).dt.date#.apply(lambda x: '-'.join(str(x)[5:].split('-')[::-1]))
coub_nk['en'] = coub_nk['Дата'].dt.to_period('W').apply(lambda r: r.end_time).dt.date#.apply(lambda x: '-'.join(str(x)[5:].split('-')[::-1]))
coub_nk['week'] = coub_nk['st'].astype(str) + ' / ' + coub_nk['en'].astype(str)
coub_nk.drop(['st','en'], axis=1,inplace=True)
coub_nk['year'] = pd.DatetimeIndex(coub_nk['Дата']).year

coub_nk_gr = coub_nk.groupby(['year','week','GUID']).count().reset_index().merge(guids_all,on='GUID',how='left')
coub_nk_gr = coub_nk_gr[~coub_nk_gr['device'].isnull()].groupby(['year','week','device'])['GUID'].count().reset_index()
coub_nk_gr['stage'] = '11.1 Стало НК 5-50 за этот период всего'
coub_nk_gr = coub_nk_gr[['year','week','stage','device','GUID']]



base_oldbitype_df = df_stages[df_stages['stage'] == '09. Подписало документы'][['year','week','device','clientID']].rename(columns={'clientID': 'cookie_id'})
base_oldbitype_df = base_oldbitype_df.merge(guids_all,on='cookie_id',how='left')

coub_nd_gr0 = base_oldbitype_df[~base_oldbitype_df['GUID'].isnull()].merge(coub_nd,on='GUID',how='left')
coub_nd_gr0 = coub_nd_gr0[~coub_nd_gr0['week_y'].isnull()].groupby(['year_x','week_x','device_x'])['cookie_id'].nunique().reset_index()
coub_nd_gr0['stage'] = '10.2 Активировано договоров из пользователей, пришедших в этот период'
coub_nd_gr0 = coub_nd_gr0[['year_x', 'week_x', 'stage','device_x', 'cookie_id' ]]

coub_nk_gr0 = base_oldbitype_df[~base_oldbitype_df['GUID'].isnull()].merge(coub_nk,on='GUID',how='left')
coub_nk_gr0 = coub_nk_gr0[~coub_nk_gr0['week_y'].isnull()].groupby(['year_x','week_x','device_x'])['cookie_id'].nunique().reset_index()
coub_nk_gr0['stage'] = '11.2 Стало НК 5-50 из пользователей, пришедших в этот период'
coub_nk_gr0 = coub_nk_gr0[['year_x', 'week_x', 'stage','device_x', 'cookie_id' ]]

ob_funnel = pd.concat([coub_nd_gr0, coub_nk_gr0], axis=0)
ob_funnel.columns = coub_nk_gr.columns
all_funnel_types = pd.concat([ob_funnel, coub_nd_gr,coub_nk_gr], axis=0, ignore_index=True).rename(columns={'GUID': 'clientID'})
all_funnel_types['type_page'] = 'Ячейка не используется'
all_funnel_types['prev_pagepath'] = 'Ячейка не используется'
all_funnel_types = all_funnel_types[['year','week','stage','type_page','prev_pagepath','device','clientID']]

Объединение кубовой и внутренней воронок и переходов

In [ ]:
df_stages_gr = pd.concat([df_stages_gr, all_funnel_types], axis=0)
df_stages_gr['agg_type'] = 'Общий (по умолчанию)'
pages_all_gr.columns = df_stages_gr.columns
df_combined = pd.concat([pages_all_gr, df_stages_gr], axis=0, ignore_index=True)
df_combined.rename(columns={'agg_type': 'widget'}, inplace=True)
df_combined[['source', 'campaign', 'widget_class', 'widget_text', 'user_level']] = pd.DataFrame([['Ячейка не используется', 'Ячейка не используется', 'Ячейка не используется', 'Ячейка не используется', 'Ячейка не используется']], index=df_combined.index)
df_combined = df_combined[['year', 'week','source', 'campaign', 'device', 'prev_pagepath','widget', 'widget_class','widget_text', 'stage', 'type_page','clientID','user_level']].fillna('Ячейка не используется')
df_combined['source'] = df_combined['widget'].astype(str)


Проверка данных

In [ ]:
temp = df_combined.groupby(['year','week','stage'])['clientID'].sum().to_frame().reset_index()
temp[temp['week'] == '2020-01-27 / 2020-02-02']

# Время прохождения этапов в воронке

In [ ]:
df_stages_time['fullVisitorId_visitId'] = df_stages_time['fullVisitorId'].astype(str) + ' | ' + df_stages_time['visitId'].astype(str)


time_df = pd.pivot_table(df_stages_time,index=['year','week','fullVisitorId_visitId'],columns='stage')['time'].reset_index().fillna(0)
time_df['step2-step1'] = (time_df[' step-2'] - time_df[' step-1']).astype(int)/1000
time_df['step3-step2'] = (time_df[' step-3'] - time_df[' step-2']).astype(int)/1000
time_df['step4-step3'] = (time_df[' step-4'] - time_df[' step-3']).astype(int)/1000
time_df['step5-step4'] = (time_df[' step-5'] - time_df[' step-4']).astype(int)/1000
time_df['step6-step5'] = (time_df[' step-6'] - time_df[' step-5']).astype(int)/1000
time_df['step7-step6'] = (time_df[' step-7'] - time_df[' step-6']).astype(int)/1000
time_df['step8-step7'] = (time_df[' step-8'] - time_df[' step-7']).astype(int)/1000
time_df['step9-step8'] = (time_df[' step-9'] - time_df[' step-8']).astype(int)/1000

#time_df['Время прохождения этапа 1 в воронке'] = (time_df[' step-5'] - time_df[' step-1']).astype(int)/1000
#time_df['Время прохождения этапа 2 в воронке'] = (time_df[' step-6'] - time_df[' step-5']).astype(int)/1000
#time_df['Время прохождения этапа 3 в воронке'] = (time_df[' step-8'] - time_df[' step-6']).astype(int)/1000

year_time = []
week_time = []
stage_time = []
value_time = []
for k in set(time_df['year']):
    for m in set(time_df['week']):
        for e in ['step2-step1', 'step3-step2', 'step4-step3', 'step5-step4','step6-step5', 'step7-step6', 'step8-step7', 'step9-step8']:
            time_df_temp = time_df[(time_df['year'] == k)&(time_df['week'] == m)]
            list_time_diff = [e for e in list(time_df_temp[e]) if e > 0]
            if len(list_time_diff) > 0: 
                avg_col = st.mean(list_time_diff)
            else:
                avg_col = 0
            year_time.append(k)
            week_time.append(m)
            stage_time.append(e)
            value_time.append(int(avg_col))
stages_time = pd.DataFrame({'year':year_time,'week':week_time,'stage':stage_time,'time':value_time})
stages_time['stage'] = stages_time['stage'].apply(lambda x: x.replace('step',''))

# Подгрузка данных в metabase

In [ ]:
import pyodbc

def multiinsert(cnxn, cursor, selector, values_list):
    for value in values_list:
        cursor.execute(selector, value)
        cnxn.commit()
        
cnxn = pyodbc.connect(Trusted_Connection='yes', driver = '{SQL Server}',server = 'xxx' , database = 'xxx')
cursor = cnxn.cursor()
dict_russian_month_reversed = {'Январь': 1,'Февраль': 2,'Март': 3,'Апрель': 4,'Май': 5,'Июнь': 6,'Июль': 7,'Август': 8,'Сентябрь': 9,'Октябрь': 10,'Ноябрь': 11,'Декабрь': 12}



In [ ]:
list_of_tuples = [tuple(x) for x in df_combined.values]

cursor.execute("DELETE FROM [MetabasePilot ].[Marketing].[DOS2_Stages_All]")
cnxn.commit()

selector = '''
INSERT INTO [MetabasePilot ].[Marketing].[DOS2_Stages_All]([year], [month], [source], [campaign], [device], [pagepath], [widget], [widget_class], [widget_text], [stage], [stage_gal], [count], [user_level])
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
'''  
values_list = list_of_tuples

multiinsert(cnxn, cursor, selector, values_list)

if pd.read_sql("SELECT * FROM [MetabasePilot ].[Marketing].[DOS2_Stages_All]", cnxn).shape == df_combined.shape:
    print('Все ок')
else:
    print('Не мэчится')

In [ ]:
list_of_tuples = [tuple(x) for x in stages_time.values]

cursor.execute("DELETE FROM [MetabasePilot ].[Marketing].[DOS2_Stages_Time]")
cnxn.commit()

selector = '''
INSERT INTO [MetabasePilot ].[Marketing].[DOS2_Stages_Time]([year], [month], [stage], [time])
VALUES (?, ?, ?, ?);
'''  
values_list = list_of_tuples

multiinsert(cnxn, cursor, selector, values_list)

if pd.read_sql("SELECT * FROM [MetabasePilot ].[Marketing].[DOS2_Stages_Time]", cnxn).shape == stages_time.shape:
    print('Все ок')
else:
    print('Не мэчится')